In [1]:
# Colabis kasutamiseks
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/thesis/')

!pip install estnltk==1.7.4
!pip install evaluate
!pip install seqeval
!pip install nervaluate

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-no

In [2]:
from modules.data_processing import DatasetProcessor
from modules.bert_data_processing import BERTDataProcessor
from modules.bert_evaluator import BERTEvaluator
ALL_TAGS = DatasetProcessor.ALL_TAGS
TAG2IDX = DatasetProcessor.TAG2IDX
IDX2TAG = DatasetProcessor.IDX2TAG
from transformers import pipeline
import numpy as np
import datetime
from transformers import AutoModelForTokenClassification
import torch
#ner_pipe = pipeline('ner', model='tartuNLP/EstBERT_NER', aggregation_strategy=None, batch_size=16)
from nervaluate import Evaluator

INFO:datasets:PyTorch version 2.5.1+cu124 available.
INFO:datasets:Polars version 1.21.0 available.
INFO:datasets:Duckdb version 1.1.3 available.
INFO:datasets:TensorFlow version 2.18.0 available.
INFO:datasets:JAX version 0.4.33 available.


INFO:config.py:54: PyTorch version 2.5.1+cu124 available.
INFO:config.py:66: Polars version 1.21.0 available.
INFO:config.py:77: Duckdb version 1.1.3 available.
INFO:config.py:112: TensorFlow version 2.18.0 available.
INFO:config.py:125: JAX version 0.4.33 available.


In [11]:
class EstBERTNEREval:
  def __init__(self, model_name, IDX2TAG):
    self.ner_pipe = pipeline('ner', model=f'tartuNLP/{model_name}', aggregation_strategy='first', batch_size=16)
    self.label2id = self.ner_pipe.model.config.label2id
    self.id2label = self.ner_pipe.model.config.id2label
    self.IDX2TAG = IDX2TAG
    self.model_name = model_name

    self.model = AutoModelForTokenClassification.from_pretrained(f'tartuNLP/{model_name}')
    self.bert_processor = BERTDataProcessor(f'tartuNLP/{model_name}')
    self.tokenizer = self.bert_processor.tokenizer
    self.tags = [self.id2label[i] for i in range(len(self.id2label))]
    self.ner_tags = list(set([label[2:] for label in self.tags if label != 'O']))
    self.evaluator = BERTEvaluator(all_tags=self.tags, ner_tags=self.ner_tags)

    self.ewt_processor = DatasetProcessor('ewt', from_json=True)
    self.ewt_test = self.ewt_processor.test#.select(range(2,5))
    self.ewt_test = self.ewt_test.map(self.convert_to_model_id)

    #self.edt_processor = DatasetProcessor('edt', from_json=True)
    #self.edt_test = self.edt_processor.test.select(range(2,7))
    #self.edt_test = self.edt_test.map(self.convert_to_model_id)

  def convert_to_model_id(self, example):
    example['tags'] = [self.IDX2TAG[tag] for tag in example['tags']]
    example['tags'] = [self.label2id.get(tag, self.label2id['O']) for tag in example['tags']]
    return example

  def process_example(self, example):
    tokens = example['tokens']
    sentence_text = ' '.join(tokens)
    # ! Siin oligi probleem. Ennustamisel peab lause andma tekstina, mitte sõnade listina
    gold_tags = [self.id2label[t] for t in example['tags']]
    predictions = self.ner_pipe(sentence_text)
    pred_tags = ["O"] * len(tokens)

    for pred in predictions:
      pred['label'] = pred.pop('entity_group')
    pred_tags = [pred for pred in predictions]
    gold_tags = bio_to_spans(tokens, gold_tags)

    #print('pred',pred_tags)
    #print('gold',gold_tags)
    # tokens = self.tokenizer.tokenize(self.tokenizer.decode(self.tokenizer.encode(sentence_text)))
    # inputs = self.tokenizer.encode(sentence_text, return_tensors="pt")
    # outputs = self.model(inputs)[0]
    # predictions = torch.argmax(outputs, dim=2)
    # prediction_list = [(token, self.tags[prediction]) for token, prediction in zip(tokens, predictions[0].tolist())]
    # prediction_list = [pair for pair in prediction_list if pair[0] not in ['[SEP]', '[UNK]', '[CLS]']]
    # for i in range(1, len(prediction_list)):
    #   if prediction_list[i][0].startswith('##'):
    #     prediction_list[i-1] = (prediction_list[i-1][0]+prediction_list[i][0][2:], prediction_list[i-1][1])

    # final_prediction_list = []
    # for i in range(len(prediction_list)):
    #   if prediction_list[i][0].startswith('##'):
    #     pass
    #   else:
    #     final_prediction_list.append(prediction_list[i])

    # print(len(final_prediction_list), final_prediction_list)

    # formatted = []
    # for pred in predictions:
    #   end = pred['start']+len(pred['word'].replace("##", ""))

    #   if pred['word'].startswith("##"):
    #     formatted[-1]['end'] = end
    #     formatted[-1]['word'] += pred['word'].replace("##", "")
    #   else:
    #     formatted.append({
    #         'start': pred['start'],
    #         'end': end,
    #         'entity': pred['entity'],
    #         'i': pred['index'],
    #         'score': pred['score'],
    #         'word': pred['word']
    #     })

    # for pred in formatted:
    #   print(pred)

    return {"gold_tags": gold_tags, "predicted_tags": pred_tags}

  def get_tags_from_dataset(self, dataset):
    gold_tags, predicted_tags = [], []

    for item in dataset:
      gold_tags.append(item['gold_tags'])
      predicted_tags.append(item['predicted_tags'])

    return gold_tags, predicted_tags

  def result_to_tsv(self, dataset, model_name, dataset_name):
        folder_name = f"results/{model_name}"
        timestamp = datetime.datetime.now().strftime("%Y-%d-%m_%H-%M")
        file_name = f"predictions_on_{dataset_name}_test_{timestamp}.tsv"
        os.makedirs(folder_name, exist_ok=True)
        file_path = os.path.join(folder_name, file_name)

        with open(file_path, "w", encoding="utf-8") as f:
            f.write("sona\tgold\tpred\n")
            #print('sona\tgold\tpred')
            for item in dataset:
                sent_tokens = item['tokens']
                preds = item['predicted_tags']
                actual_labels = item['gold_tags']
                print('tokens', len(sent_tokens), sent_tokens)
                print('predictions', len(preds), preds)
                print('gold', len(actual_labels), actual_labels)
                assert len(sent_tokens) == len(preds)
                assert len(preds) == len(actual_labels)
                for w, actual_tag, predicted_tag in zip(sent_tokens, actual_labels, preds):
                    f.write(f'{w}\t{actual_tag}\t{predicted_tag}\n')
                    #print(f'{w}\t{actual_tag}\t{predicted_tag}')
                f.write("\n")
                    #print()


  def evaluate(self):
    self.ewt_test = self.ewt_test.map(self.process_example)
    #self.edt_test = self.edt_test.map(self.process_example)

    ewt_gold_tags, ewt_predicted_tags = self.get_tags_from_dataset(self.ewt_test)
    evaluator = Evaluator(ewt_gold_tags, ewt_predicted_tags, tags=self.ner_tags)
    results, results_per_tag, result_indices, result_indices_by_tag = evaluator.evaluate()
    print(results['strict'])
    for k in results_per_tag:
      print(k, results_per_tag[k]['strict'])
    #edt_gold_tags, edt_predicted_tags = self.get_tags_from_dataset(self.edt_test)

    #combined_gold_tags = ewt_gold_tags + edt_gold_tags
    #combined_predicted_tags = ewt_predicted_tags + edt_predicted_tags

    #ewt_nervaluate, ewt_by_tag, _, _ = self.evaluator.get_nervaluate_results(ewt_predicted_tags, ewt_gold_tags)
    #edt_nervaluate, edt_by_tag, _, _ = self.evaluator.get_nervaluate_results(edt_predicted_tags, edt_gold_tags)
    #combined_nervaluate, combined_by_tag, _, _ = self.evaluator.get_nervaluate_results(combined_predicted_tags, combined_gold_tags)

    #self.evaluator.evaluation_to_json(ewt_nervaluate['strict'], ewt_by_tag, self.model_name, trained_on=None, evaluated_on="EWT")
    #self.evaluator.evaluation_to_json(edt_nervaluate['strict'], edt_by_tag, self.model_name, trained_on=None, evaluated_on="EDT")
    #self.evaluator.evaluation_to_json(combined_nervaluate['strict'], combined_by_tag, self.model_name, trained_on=None, evaluated_on="Combined")

    #self.result_to_tsv(self.ewt_test, self.model_name, "EWT")
    #self.result_to_tsv(self.edt_test, self.model_name, "EDT")


In [12]:
estbertner = EstBERTNEREval("EstBERT_NER", IDX2TAG)
estbertner.evaluate()

Some weights of the model checkpoint at tartuNLP/EstBERT_NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
Some weights of the model checkpoint at tartuNLP/EstBERT_NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with ano

Map:   0%|          | 0/913 [00:00<?, ? examples/s]

Map:   0%|          | 0/913 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'correct': 270, 'incorrect': 61, 'partial': 0, 'missed': 220, 'spurious': 105, 'possible': 551, 'actual': 436, 'precision': 0.6192660550458715, 'recall': 0.4900181488203267, 'f1': 0.5471124620060789}
LOC {'correct': 10, 'incorrect': 9, 'partial': 0, 'missed': 9, 'spurious': 48, 'possible': 28, 'actual': 67, 'precision': 0.14925373134328357, 'recall': 0.35714285714285715, 'f1': 0.21052631578947364}
PER {'correct': 215, 'incorrect': 26, 'partial': 0, 'missed': 190, 'spurious': 26, 'possible': 431, 'actual': 267, 'precision': 0.8052434456928839, 'recall': 0.4988399071925754, 'f1': 0.6160458452722062}
ORG {'correct': 45, 'incorrect': 26, 'partial': 0, 'missed': 21, 'spurious': 31, 'possible': 92, 'actual': 102, 'precision': 0.4411764705882353, 'recall': 0.4891304347826087, 'f1': 0.46391752577319595}


In [13]:
estbertner.IDX2TAG

{0: 'O',
 1: 'I-UNK',
 2: 'I-PROD',
 3: 'I-PER',
 4: 'I-ORG',
 5: 'I-MUU',
 6: 'I-LOC',
 7: 'I-GEP',
 8: 'I-EVE',
 9: 'B-UNK',
 10: 'B-PROD',
 11: 'B-PER',
 12: 'B-ORG',
 13: 'B-MUU',
 14: 'B-LOC',
 15: 'B-GEP',
 16: 'B-EVE'}

In [14]:
v2IDX2TAG = {0: 'O',
 1: 'I-UNK',
 2: 'I-PROD',
 3: 'I-PER',
 4: 'I-ORG',
 5: 'I-MUU',
 6: 'I-LOC',
 7: 'I-GPE',
 8: 'I-EVENT',
 9: 'B-UNK',
 10: 'B-PROD',
 11: 'B-PER',
 12: 'B-ORG',
 13: 'B-MUU',
 14: 'B-LOC',
 15: 'B-GPE',
 16: 'B-EVENT'}
estbertnerv2 = EstBERTNEREval("EstBERT_NER_V2", v2IDX2TAG)


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/410k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


Map:   0%|          | 0/913 [00:00<?, ? examples/s]

In [15]:
estbertnerv2.evaluate()

Map:   0%|          | 0/913 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'correct': 391, 'incorrect': 154, 'partial': 0, 'missed': 175, 'spurious': 264, 'possible': 720, 'actual': 809, 'precision': 0.48331273176761436, 'recall': 0.5430555555555555, 'f1': 0.5114453891432309}
EVENT {'correct': 8, 'incorrect': 11, 'partial': 0, 'missed': 3, 'spurious': 11, 'possible': 22, 'actual': 30, 'precision': 0.26666666666666666, 'recall': 0.36363636363636365, 'f1': 0.30769230769230765}
ORG {'correct': 37, 'incorrect': 32, 'partial': 0, 'missed': 23, 'spurious': 37, 'possible': 92, 'actual': 106, 'precision': 0.3490566037735849, 'recall': 0.40217391304347827, 'f1': 0.3737373737373737}
PER {'correct': 290, 'incorrect': 45, 'partial': 0, 'missed': 96, 'spurious': 34, 'possible': 431, 'actual': 369, 'precision': 0.7859078590785907, 'recall': 0.6728538283062645, 'f1': 0.7250000000000001}
GPE {'correct': 35, 'incorrect': 2, 'partial': 0, 'missed': 4, 'spurious': 11, 'possible': 41, 'actual': 48, 'precision': 0.7291666666666666, 'recall': 0.8536585365853658, 'f1': 0.786516853

In [6]:
def bio_to_spans(tokens, tags):
    """
    Convert BIO tagged tokens to entity spans.

    Args:
        tokens (list): List of tokens
        tags (list): List of BIO tags

    Returns:
        list: List of entity dictionaries with group, word, start and end positions
    """
    entities = []
    current_entity = None
    char_index = 0

    # Keep track of character positions
    token_char_spans = []
    for token in tokens:
        token_char_spans.append((char_index, char_index + len(token)))
        # Add 1 for the space after each token (except the last one)
        char_index += len(token) + 1

    for i, (token, tag) in enumerate(zip(tokens, tags)):
        if tag.startswith('B-'):
            # If we were tracking an entity, add it to the list
            if current_entity:
                entities.append(current_entity)

            # Start a new entity
            entity_group = tag[2:]  # Remove 'B-' prefix
            start_char = token_char_spans[i][0]
            current_entity = {
                'label': entity_group,
                #'word': token,
                'start': start_char,
                'end': token_char_spans[i][1]
            }

        elif tag.startswith('I-') and current_entity:
            # Continue the current entity if the tags match
            if current_entity['label'] == tag[2:]:
                # Extend the word and end position
                #current_entity['word'] += ' ' + token
                current_entity['end'] = token_char_spans[i][1]

        elif tag == 'O':
            # End of an entity
            if current_entity:
                entities.append(current_entity)
                current_entity = None

    # Add the last entity if there is one
    if current_entity:
        entities.append(current_entity)

    return entities

# Example usage
tokens = ['Jürgen', 'Zopp', 'kaotas', 'Petangeis', '64000', 'euro', 'suuruse', 'auhinnafondiga', 'Challenger-turniiri', 'poolfinaalis', 'prantslasele', 'Paul-Henri', 'Mathieule', '6:7', ',', '2:6', '.']
tags = ['B-PER', 'I-PER', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O']

spans = bio_to_spans(tokens, tags)
for span in spans:
    print(span)

{'label': 'PER', 'start': 0, 'end': 11}
{'label': 'LOC', 'start': 19, 'end': 28}
{'label': 'PER', 'start': 109, 'end': 129}


In [127]:
pipe = pipeline('ner', model='tartuNLP/EstBERT_NER')
#pipe("Erinevus Best Pressi välja antud Roger Federeri elulooraamatust on selles, et Nadal osales treeningus.")
orig_tokens = ['Erinevus', 'Best', 'Pressi', 'välja', 'antud', 'Roger', 'Federeri', '(', 'Luksemburg', ')', 'elulooraamatust', 'on', 'selles', ',', 'et', '.']
sentence = ' '.join(orig_tokens)

from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

model = AutoModelForTokenClassification.from_pretrained("tartuNLP/EstBERT_NER")
tokenizer = AutoTokenizer.from_pretrained("tartuNLP/EstBERT_NER")
label_list = [label for label in model.config.label2id.keys()]
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sentence)))
inputs = tokenizer.encode(sentence, return_tensors="pt")

outputs = model(inputs)[0]
preds = torch.argmax(outputs, dim=2)

preds_list = [(token, label_list[prediction]) for token, prediction in zip(tokens, preds[0].tolist())][1:]
preds_list = [p for p in preds_list if p[0] != '[SEP]']
preds_list = [p for p in preds_list if not p[0].startswith("##")]
print(preds_list)
print(orig_tokens)
pipe(sentence)

Some weights of the model checkpoint at tartuNLP/EstBERT_NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Some weights of the model checkpoint at tartuNLP/EstBERT_NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with anothe

[('Eri', 'O'), ('Best', 'B-ORG'), ('Press', 'I-ORG'), ('välja', 'O'), ('antud', 'O'), ('Roger', 'B-PER'), ('Fede', 'I-PER'), ('(', 'O'), ('Luksembur', 'B-LOC'), (')', 'O'), ('eluloo', 'O'), ('on', 'O'), ('selles', 'O'), (',', 'O'), ('et', 'O'), ('.', 'O')]
['Erinevus', 'Best', 'Pressi', 'välja', 'antud', 'Roger', 'Federeri', '(', 'Luksemburg', ')', 'elulooraamatust', 'on', 'selles', ',', 'et', '.']


In [6]:
from transformers import BertForTokenClassification, Trainer, TrainingArguments
from modules.bert_data_processing import BERTDataProcessor

def pipe(model_name, dataset_name):
  model = BertForTokenClassification.from_pretrained(f'tartuNLP/{model_name}')
  bert_processor = BERTDataProcessor(f'tartuNLP/{model_name}')

  label2id = model.config.label2id

  def convert_to_model_id(example):
    example['tags'] = [IDX2TAG[tag] for tag in example['tags']]
    example['tags'] = [label2id.get(tag, label2id['O']) for tag in example['tags']]
    return example

  dataset_processor = DatasetProcessor(dataset_name, from_json=True)
  test = dataset_processor.test

  test = test.map(convert_to_model_id)
  test = bert_processor.tokenize_dataset(test)

  tags = [model.config.id2label[i] for i in range(len(model.config.id2label))]
  ner_tags = list(set([label[2:] for label in tags if label != 'O']))
  evaluator = BERTEvaluator(all_tags=tags, ner_tags=ner_tags)

  training_args = TrainingArguments(
            report_to='none',
            output_dir='./results',
            #per_device_eval_batch_size=16
        )
  trainer = Trainer(
            model=model,
            processing_class=bert_processor.tokenizer,
            data_collator=bert_processor.data_collator,
            compute_metrics=evaluator.compute_metrics,
            args=training_args
        )

  preds, labels = evaluator.get_predictions(test,trainer)
  res, tag, _, _ = evaluator.get_nervaluate_results(preds,labels)
  print(res['strict'])
  # for s in test:
  #   print(s)
  # preds, labels = evaluator.get_predictions(test, trainer)
  # for s, pred, label in zip(test, preds, labels):
  #   sent_tokens = bert_processor.tokenizer.convert_ids_to_tokens(s['input_ids'])[1:]
  #   sent_tokens = [w for w in sent_tokens if w != '[PAD]' and w != '[SEP]']
  #   print(len(sent_tokens), sent_tokens)
  #   print(len(pred), pred)
  #   print(len(label), label)



In [7]:
pipe("EstBERT_NER", 'ewt')

Some weights of the model checkpoint at tartuNLP/EstBERT_NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Map:   0%|          | 0/913 [00:00<?, ? examples/s]

Map:   0%|          | 0/913 [00:00<?, ? examples/s]

{'correct': 307, 'incorrect': 253, 'partial': 0, 'missed': 821, 'spurious': 118, 'possible': 1381, 'actual': 678, 'precision': 0.4528023598820059, 'recall': 0.222302679217958, 'f1': 0.2982030111704711}


In [ ]:
import os
import json
import datetime

timestamp = datetime.datetime.now().strftime("%Y-%d-%m_%H-%M")
nervaluate_results, nervaluate_by_tag, _, _ = evaluator.get_nervaluate_results(true_predictions, true_labels)

def evaluation_to_json(nervaluate_strict_overall, nervaluate_by_tag, model_name, trained_on, evaluated_on):
  results = {}
  results["strict"] = nervaluate_strict_overall
  for key in nervaluate_by_tag:
    results[key] = nervaluate_by_tag[key]['strict']

  formatted_output = {
      "model": model_name,
      "trained_on": trained_on,
      "evaluated_on": evaluated_on,
      "results": results
  }

  folder_name = f"results/{model_name}_{trained_on}"
  timestamp = datetime.datetime.now().strftime("%Y-%d-%m_%H-%M")
  file_name = f"eval_on_{evaluated_on}_test_{timestamp}.json"

  os.makedirs(folder_name, exist_ok=True)
  file_path = os.path.join(folder_name, file_name)
  with open(file_path, "w", encoding="utf-8") as f:
    json.dump(formatted_output, f, indent=4)

  print(f"Salvestasin: {file_path}")

evaluation_to_json(nervaluate_results['strict'], nervaluate_by_tag,"estbert","ewt","ewt")